In [1]:
!pip install pandas xlrd ipdb

In [2]:
import os
import pandas as pd
import ipdb
from datetime import datetime, time

In [3]:
%ls data

weekly-2020-02-10 0925.xlsx


In [4]:
libs = sorted(pd.read_csv('libraries.csv').full_name.tolist())

In [5]:
# CONVERT
dn = 'data'
fn = dn + '/' + 'weekly-2020-02-10 0925.xlsx'
xl = pd.ExcelFile(fn)
sheets = xl.sheet_names
res=[]
for sheet in sheets:
    df = xl.parse(sheet)
    date = datetime.strptime(df.columns[0], '%d %B %Y')
    df = (
        df.loc[df['Counter Name'].isin(libs)]
        .rename(columns={df.columns[0]: 'dt'})
        .fillna('x')        
    )
    times=df.loc[df.dt!='x']
    df.dt = df.dt.replace(to_replace='x', method='ffill')
    dt = list(map(lambda x: datetime.combine(date, time(int(x[:2]), 0)), df.dt.tolist()))
    df.dt = dt
    res.append(df)                             

df = pd.concat(res)
df = df.rename(columns={'Counter Name': 'Library'})

In [6]:
def to_hours(x):
    return pd.to_datetime(x).hour

oh = pd.read_csv('opening hours.csv', converters={'start': lambda x: to_hours(x), 'finish': lambda x: to_hours(x)})
oh

,library,day,start,finish,opening_type
0,Burnt Oak,Mon,14,20,staffed
1,Burnt Oak,Wed,9,12,staffed
2,Burnt Oak,Fri,14,17,staffed
3,Burnt Oak,Sat,14,17,staffed
4,Burnt Oak,Mon,9,14,SSO
...,...,...,...,...,...
144,South Friern,Sat,14,17,staffed
145,North Finchley,Thu,12,20,SSO
146,North Finchley,Fri,17,20,SSO
147,East Finchley,Thu,17,20,SSO


In [ ]:
# ADD OPENING TYPE

def get_ot(dt, lib):
    in_df = pd.concat([dt, lib], axis=1)
    res=[]
    for row in in_df.itertuples():
        day = row.dt.day_name()[:3]
        mask = (
            (oh.library==row.Library) &
            (oh.day==day) &
            (oh.start<=row.dt.hour) &
            (oh.finish>row.dt.hour)
        )
        if oh.loc[mask].empty:
            res.append('closed')
        else:
            ot = oh.loc[mask].opening_type.values[0]
            res.append(ot)
#     ipdb.set_trace()
    return pd.Series(res)

ots = get_ot(df.dt, df.Library)
df['opening_type'] = ots.values
df

In [ ]:
from index import get_df_with_opening_hours

In [ ]:
%ls data
fn = 'data/weekly-2020-02-10 0925.xlsx'
check = get_df_with_opening_hours(fn)
check